Aula 1 - Análise sintática ascendente


In [3]:
#@title Análise ascendente (SHIFT/REDUCE) com erros claros — 1 célula
import re
from dataclasses import dataclass

# ---------- util p/ mensagem com ^ ----------
def show_error(src, pos, msg):
    line_start = src.rfind('\n', 0, pos) + 1
    line_end = src.find('\n', pos)
    if line_end == -1: line_end = len(src)
    line = src[line_start:line_end]
    col = max(0, pos - line_start)
    print(f"Erro de sintaxe na coluna {col+1}: {msg}")
    print(line)
    print(' ' * col + '^')

class SynErr(Exception):
    def __init__(self, pos, msg): super().__init__(pos, msg)

# ---------- lexer ----------
@dataclass
class Tok:
    kind: str   # 'NUM','ID','OP','LP','RP','EOF'
    lex: str
    pos: int

_tok = re.compile(r"""
    (?P<NUM>\d+)
  | (?P<ID>[A-Za-z_]\w*)
  | (?P<OP>[+\-*/])
  | (?P<LP>\()
  | (?P<RP>\))
  | (?P<WS>\s+)
""", re.VERBOSE)

def lex(src: str):
    out=[]
    for m in _tok.finditer(src):
        k=m.lastgroup
        if k=='WS':
            continue
        if k in ('NUM','ID','OP','LP','RP'):
            out.append(Tok(k, m.group(), m.start()))
        else:
            raise SynErr(m.start(), f"caractere inválido: {m.group()!r}")
    out.append(Tok('EOF','', len(src)))
    return out

# ---------- AST ----------
class AST: ...
@dataclass
class Num(AST): value:int
@dataclass
class Id(AST):  name:str
@dataclass
class Bin(AST): op:str; left:AST; right:AST

def to_infix(n:AST)->str:
    if isinstance(n, Num): return str(n.value)
    if isinstance(n, Id):  return n.name
    if isinstance(n, Bin): return f"({to_infix(n.left)} {n.op} {to_infix(n.right)})"
    return "<?>"

# ---------- parser ascendente (precedência de operadores) ----------
PREC = {'+':1, '-':1, '*':2, '/':2}
LEFT = {'+':True, '-':True, '*':True, '/':True}

def reduce_once(vals, ops, log):
    op = ops.pop()
    b = vals.pop(); a = vals.pop()
    node = Bin(op, a, b)
    vals.append(node)
    log.append(("REDUCE", op, stack_view(vals), ops_view(ops)))

def stack_view(vals): return "[" + ", ".join(to_infix(v) for v in vals) + "]"
def ops_view(ops):    return "[" + " ".join(ops) + "]"

def parse_bottom_up(src:str, verbose=True):
    tokens = lex(src)
    vals, ops, log = [], [], []
    prev_kind = None

    def shift(item): log.append(("SHIFT", item, stack_view(vals), ops_view(ops)))

    i=0
    while i < len(tokens):
        t = tokens[i]
        if t.kind == 'NUM':
            vals.append(Num(int(t.lex))); shift(t.lex); prev_kind = 'NUM'
        elif t.kind == 'ID':
            vals.append(Id(t.lex)); shift(t.lex); prev_kind = 'ID'
        elif t.kind == 'LP':
            ops.append('('); shift('('); prev_kind = 'LP'
        elif t.kind == 'RP':
            # ) não pode vir logo após OP ou LP
            if prev_kind in ('OP','LP', None):
                raise SynErr(t.pos, "operando ausente antes de ')'")
            while ops and ops[-1] != '(':
                reduce_once(vals, ops, log)
            if not ops:
                raise SynErr(t.pos, "faltou '(' antes de ')'")
            ops.pop(); shift(')'); prev_kind = 'RP'
        elif t.kind == 'OP':
            # dois operadores em sequência → erro didático
            if prev_kind in ('OP','LP', None):
                raise SynErr(t.pos, f"operando ausente antes de `{t.lex}`")
            while ops and ops[-1] in PREC and (
                PREC[ops[-1]] > PREC[t.lex] or
                (PREC[ops[-1]] == PREC[t.lex] and LEFT[t.lex])
            ):
                reduce_once(vals, ops, log)
            ops.append(t.lex); shift(t.lex); prev_kind = 'OP'
        elif t.kind == 'EOF':
            break
        else:
            raise SynErr(t.pos, f"token inválido: {t.kind}")
        i += 1

    # fim de entrada: reduzir o que restar
    while ops:
        if ops[-1] == '(':
            raise SynErr(len(src), "esperado ')' no final")
        if len(vals) < 2:
            raise SynErr(len(src), "expressão incompleta (faltou operando)")
        reduce_once(vals, ops, log)

    if len(vals) != 1:
        raise SynErr(len(src), "expressão incompleta")

    if verbose:
        print("Passos (ação, item, pilha-valores, pilha-operadores):")
        for k, item, sv, ov in log:
            print(f"{k:7} {item:>3}  {sv:>22}  ops={ov}")
        print("\nAST:", vals[0])
        print("Infixo:", to_infix(vals[0]))
    return vals[0]

# ---------- demonstrações ----------
def demo():
    cases = [
        "2 + 3 * 4",        # ok: * tem maior precedência
        "(2 + 3) * 4",      # ok: parênteses
        "x * (y + 1) - 5",  # ok: ids + nums
        "2 + * 3",          # erro: operando ausente antes de '*'
        "(2 + 3 * 4"        # erro: esperado ')'
    ]
    for s in cases:
        print("="*60)
        print("Entrada:", s)
        try:
            parse_bottom_up(s, verbose=True)
        except SynErr as e:
            pos, msg = e.args
            show_error(s, pos, msg)
        except Exception as e:
            print("Erro inesperado:", e)

# Execute:
demo()


Entrada: 2 + 3 * 4
Passos (ação, item, pilha-valores, pilha-operadores):
SHIFT     2                     [2]  ops=[]
SHIFT     +                     [2]  ops=[+]
SHIFT     3                  [2, 3]  ops=[+]
SHIFT     *                  [2, 3]  ops=[+ *]
SHIFT     4               [2, 3, 4]  ops=[+ *]
REDUCE    *            [2, (3 * 4)]  ops=[+]
REDUCE    +         [(2 + (3 * 4))]  ops=[]

AST: Bin(op='+', left=Num(value=2), right=Bin(op='*', left=Num(value=3), right=Num(value=4)))
Infixo: (2 + (3 * 4))
Entrada: (2 + 3) * 4
Passos (ação, item, pilha-valores, pilha-operadores):
SHIFT     (                      []  ops=[(]
SHIFT     2                     [2]  ops=[(]
SHIFT     +                     [2]  ops=[( +]
SHIFT     3                  [2, 3]  ops=[( +]
REDUCE    +               [(2 + 3)]  ops=[(]
SHIFT     )               [(2 + 3)]  ops=[]
SHIFT     *               [(2 + 3)]  ops=[*]
SHIFT     4            [(2 + 3), 4]  ops=[*]
REDUCE    *         [((2 + 3) * 4)]  ops=[]

AST: Bin(o

Aula 2 - Diferenças entre LR, LALR e SLR

In [ ]:
#@title Diferenças entre SLR, LALR e LR(1): conflitos e tamanho das tabelas (1 célula)
from dataclasses import dataclass
from collections import defaultdict, deque

EPS = 'ε'
END = '$'

# ---------- Estruturas ----------
@dataclass(frozen=True)
class Prod:
    head: str
    body: tuple

@dataclass(frozen=True)
class LR0Item:
    head: str; body: tuple; dot: int

@dataclass(frozen=True)
class LR1Item:
    head: str; body: tuple; dot: int; la: str

# ---------- Montagem de gramática ----------
def build_grammar(rules: dict, start: str):
    nonterms = set(rules.keys())
    prods = [Prod(A, tuple(rhs)) for A, rhss in rules.items() for rhs in rhss]
    symbols = {s for p in prods for s in p.body}
    terms = {s for s in symbols if s not in nonterms and s != EPS}
    startp = Prod(start+"'", (start,))  # S' → S
    return prods, nonterms, terms, startp

# ---------- FIRST / FOLLOW ----------
def compute_first(prods, nonterms, terms):
    FIRST = {X:set() for X in nonterms|terms|{EPS,END}}
    FIRST[EPS] = {EPS}; FIRST[END] = {END}
    for t in terms: FIRST[t].add(t)

    changed = True
    while changed:
        changed = False
        for A, body in [(p.head, p.body) for p in prods]:
            add, nullable = set(), True if not body else False
            if not body: add |= {EPS}
            else:
                for X in body:
                    add |= (FIRST[X] - {EPS})
                    if EPS in FIRST[X]: nullable = True; continue
                    nullable = False; break
                if nullable: add.add(EPS)
            if not add.issubset(FIRST[A]): FIRST[A] |= add; changed = True

    def first_seq(seq):
        out = set()
        if not seq: out.add(EPS); return out
        for X in seq:
            out |= (FIRST[X]-{EPS})
            if EPS in FIRST[X]: continue
            return out
        out.add(EPS); return out
    return FIRST, first_seq

def compute_follow(prods, nonterms, terms, start):
    FOLLOW = {A:set() for A in nonterms}
    FOLLOW[start].add(END)
    FIRST, first_seq = compute_first(prods, nonterms, terms)
    changed = True
    while changed:
        changed = False
        for p in prods:
            A, rhs = p.head, p.body
            for i,X in enumerate(rhs):
                if X in nonterms:
                    beta = list(rhs[i+1:])
                    f = first_seq(beta)
                    before = set(FOLLOW[X])
                    FOLLOW[X] |= (f - {EPS})
                    if not beta or EPS in f:
                        FOLLOW[X] |= FOLLOW[A]
                    changed |= (before != FOLLOW[X])
    return FOLLOW, FIRST, first_seq

# ---------- Fechos e GOTO ----------
def closure_lr0(I, prods_by_head, nonterms):
    I = set(I); changed = True
    while changed:
        changed = False
        for it in list(I):
            if it.dot < len(it.body):
                B = it.body[it.dot]
                if B in nonterms:
                    for p in prods_by_head[B]:
                        newi = LR0Item(B, p.body, 0)
                        if newi not in I: I.add(newi); changed = True
    return frozenset(I)

def goto_lr0(I, X, pb, nonterms):
    J = {LR0Item(it.head, it.body, it.dot+1) for it in I if it.dot < len(it.body) and it.body[it.dot]==X}
    return closure_lr0(J, pb, nonterms) if J else frozenset()

def closure_lr1(I, prods_by_head, nonterms, first_seq):
    I = set(I); changed = True
    while changed:
        changed = False
        for it in list(I):
            if it.dot < len(it.body):
                B = it.body[it.dot]
                if B in nonterms:
                    beta = list(it.body[it.dot+1:])
                    LA = first_seq(beta+[it.la]) - {EPS}
                    for p in prods_by_head[B]:
                        for a in LA:
                            newi = LR1Item(B, p.body, 0, a)
                            if newi not in I: I.add(newi); changed = True
    return frozenset(I)

def goto_lr1(I, X, pb, nonterms, first_seq):
    J = {LR1Item(it.head, it.body, it.dot+1, it.la) for it in I if it.dot < len(it.body) and it.body[it.dot]==X}
    return closure_lr1(J, pb, nonterms, first_seq) if J else frozenset()

# ---------- Conjuntos canônicos ----------
def C_lr0(prods, nonterms, terms, startp):
    pb = defaultdict(list)
    for p in prods+[startp]: pb[p.head].append(p)
    I0 = closure_lr0({LR0Item(startp.head, startp.body, 0)}, pb, nonterms)
    C, Q = [I0], deque([I0]); trans = []
    symbols = list(terms|nonterms)
    while Q:
        I = Q.popleft()
        for X in symbols:
            J = goto_lr0(I, X, pb, nonterms)
            if not J: continue
            if J not in C: C.append(J); Q.append(J)
            trans.append((I,X,J))
    return C, trans

def C_lr1(prods, nonterms, terms, start, startp, first_seq):
    pb = defaultdict(list)
    for p in prods+[startp]: pb[p.head].append(p)
    I0 = closure_lr1({LR1Item(startp.head, startp.body, 0, END)}, pb, nonterms, first_seq)
    C, Q = [I0], deque([I0]); trans = []
    symbols = list(terms|nonterms)
    while Q:
        I = Q.popleft()
        for X in symbols:
            J = goto_lr1(I, X, pb, nonterms, first_seq)
            if not J: continue
            if J not in C: C.append(J); Q.append(J)
            trans.append((I,X,J))
    return C, trans

# ---------- Tabelas e conflitos ----------
def table_slr(prods, nonterms, terms, start, startp):
    C, trans = C_lr0(prods, nonterms, terms, startp)
    FOLLOW, _, _ = compute_follow(prods, nonterms, terms, start)
    idx = {I:i for i,I in enumerate(C)}
    ACTION = defaultdict(lambda: defaultdict(list)); GOTO = defaultdict(dict)

    for I,X,J in trans:
        if X in terms: ACTION[idx[I]][X].append(('s', idx[J]))
        else: GOTO[idx[I]][X] = idx[J]

    for I in C:
        i = idx[I]
        for it in I:
            if it.dot == len(it.body):
                if it.head == startp.head and it.body == startp.body:
                    ACTION[i][END].append(('acc', None))
                else:
                    for a in FOLLOW[it.head]:
                        ACTION[i][a].append(('r', Prod(it.head, it.body)))

    conflicts = []
    for i,row in ACTION.items():
        for a,acts in row.items():
            if len(acts) > 1: conflicts.append((i,a,acts))
    return ACTION, GOTO, conflicts, len(C)

def table_lr1(prods, nonterms, terms, start, startp):
    FIRST, first_seq = compute_first(prods, nonterms, terms)
    C, trans = C_lr1(prods, nonterms, terms, start, startp, first_seq)
    idx = {I:i for i,I in enumerate(C)}
    ACTION = defaultdict(lambda: defaultdict(list)); GOTO = defaultdict(dict)

    for I,X,J in trans:
        if X in terms: ACTION[idx[I]][X].append(('s', idx[J]))
        else: GOTO[idx[I]][X] = idx[J]

    for I in C:
        i = idx[I]
        for it in I:
            if it.dot == len(it.body):
                if it.head == startp.head and it.body == startp.body and it.la == END:
                    ACTION[i][END].append(('acc', None))
                else:
                    ACTION[i][it.la].append(('r', Prod(it.head, it.body)))

    conflicts = []
    for i,row in ACTION.items():
        for a,acts in row.items():
            if len(acts) > 1: conflicts.append((i,a,acts))
    return ACTION, GOTO, conflicts, len(C), first_seq

def table_lalr1(prods, nonterms, terms, start, startp):
    # monta LR(1) e depois funde estados com mesmo núcleo LR(0)
    ACT1, G1, _, n_lr1, first_seq = table_lr1(prods, nonterms, terms, start, startp)
    pb = defaultdict(list)
    for p in prods+[startp]: pb[p.head].append(p)
    C, trans = C_lr1(prods, nonterms, terms, start, startp, first_seq)
    cores = [frozenset((it.head,it.body,it.dot) for it in I) for I in C]
    unique_cores = []
    core_id = {}
    for i,c in enumerate(cores):
        if c not in core_id:
            core_id[c] = len(unique_cores)
            unique_cores.append(c)
    old2new = {i:core_id[cores[i]] for i in range(len(C))}
    ACTION = defaultdict(lambda: defaultdict(list)); GOTO = defaultdict(dict)

    # shifts
    idx = {I:i for i,I in enumerate(C)}
    for I,X,J in trans:
        im, jm = old2new[idx[I]], old2new[idx[J]]
        if X in terms: ACTION[im][X].append(('s', jm))
        else: GOTO[im][X] = jm

    # reduções (une lookaheads no estado fundido)
    merged_items = defaultdict(set)
    for old_i, I in enumerate(C):
        im = old2new[old_i]; merged_items[im] |= set(I)

    startp_item = Prod(startp.head, startp.body)
    for im, items in merged_items.items():
        for it in items:
            if it.dot == len(it.body):
                if it.head == startp.head and it.body == startp.body and it.la == END:
                    ACTION[im][END].append(('acc', None))
                else:
                    ACTION[im][it.la].append(('r', Prod(it.head, it.body)))

    def conflict_kind(acts):
        kinds = {k for k,_ in acts}
        if 's' in kinds and 'r' in kinds: return 'shift/reduce'
        if sum(1 for k,_ in acts if k=='r') >= 2: return 'reduce/reduce'
        return 'outro'

    conflicts = []
    for i,row in ACTION.items():
        for a,acts in row.items():
            # remove duplicatas exatas
            uniq=[]; seen=set()
            for act in acts:
                if act not in seen: uniq.append(act); seen.add(act)
            if len(uniq) > 1:
                conflicts.append((i,a,uniq, conflict_kind(uniq)))

    return ACTION, GOTO, conflicts, len(unique_cores), n_lr1

# ---------- Relatórios curtos ----------
def print_conflicts(title, confs, max_show=3):
    print(title, "→", "nenhum conflito" if not confs else f"{len(confs)} conflito(s)")
    for i,(st,a,acts,*rest) in enumerate(confs[:max_show],1):
        kinds = {k for k,_ in acts}
        kind = "shift/reduce" if ('s' in kinds and 'r' in kinds) else ("reduce/reduce" if sum(1 for k,_ in acts if k=='r')>=2 else "outro")
        print(f"  #{i}: estado {st}, lookahead `{a}` → {kind}")

# ---------- Gramática G1: SLR falha; LALR/LR(1) OK ----------
G1 = {
    'S': [['L','=','R'], ['R']],
    'L': [['*','R'], ['id']],
    'R': [['L']]
}
p1, N1, T1, S1p = build_grammar(G1, 'S')
slrA, slrG, slrC, n_slr = table_slr(p1, N1, T1, 'S', S1p)
lalrA, lalrG, lalrC, n_lalr, n_lr1_dummy = table_lalr1(p1, N1, T1, 'S', S1p)
lr1A, lr1G, lr1C, n_lr1, _ = table_lr1(p1, N1, T1, 'S', S1p)

print("=== G1: S→L=R | R; L→*R | id; R→L ===")
print_conflicts("SLR(1)", slrC)
print_conflicts("LALR(1)", [(s,a,acts) for (s,a,acts,_) in lalrC])
print_conflicts("LR(1) ", lr1C)
print(f"Estados — SLR(LR0): {n_slr}, LALR: {n_lalr}, LR(1): {n_lr1}\n")

# ---------- Gramática G2: Expressões (mostra tamanhos) ----------
G2 = {
    'E': [['E','+','T'], ['T']],
    'T': [['T','*','F'], ['F']],
    'F': [['(','E',')'], ['id']]
}
p2, N2, T2, S2p = build_grammar(G2, 'E')
_, _, _, n_slr2 = table_slr(p2, N2, T2, 'E', S2p)
_, _, lalrC2, n_lalr2, n_lr12 = table_lalr1(p2, N2, T2, 'E', S2p)
_, _, lr1C2, n_lr12_exact, _ = table_lr1(p2, N2, T2, 'E', S2p)

print("=== G2: E→E+T|T; T→T*F|F; F→(E)|id ===")
print_conflicts("SLR(1)", [])
print_conflicts("LALR(1)", [(s,a,acts) for (s,a,acts,_) in lalrC2])
print_conflicts("LR(1) ", lr1C2)
print(f"Estados — SLR(LR0): {n_slr2}, LALR: {n_lalr2}, LR(1): {n_lr12_exact}")
print("\nNota: LALR(1) funde estados do LR(1) → menos estados, mesma linguagem (neste caso).")


Aula 3 - Criando parser com Bison no Colab

In [7]:
#@title Bison no Colab (silencia avisos + erro verboso) — 1 célula
%%bash
set -e

# 1) Instalar (silenciando avisos)
apt-get -qq update 2>/dev/null
apt-get -qq install -y bison build-essential >/dev/null 2>&1

# 2) Arquivo calc.y
cat > calc.y << 'EOF'
%{
  #include <stdio.h>
  #include <stdlib.h>
  void yyerror(const char *s);
  int yylex(void);
%}

%define parse.error verbose        /* mensagens tipo: unexpected '*', expecting NUM or '(' */
%define api.value.type {int}
%token NUM
%left '+' '-'
%left '*' '/'
%start input

%%
/* múltiplas linhas; cada linha pode ter uma expressão */
input
  : /* vazio */
  | input '\n'
  | input expr '\n'          { printf("%d\n", $2); }
  ;

expr
  : expr '+' expr            { $$ = $1 + $3; }
  | expr '-' expr            { $$ = $1 - $3; }
  | expr '*' expr            { $$ = $1 * $3; }
  | expr '/' expr            { if ($3==0) { yyerror("divisão por zero"); YYERROR; } else $$ = $1 / $3; }
  | '(' expr ')'             { $$ = $2; }
  | NUM                      { $$ = $1; }
  ;
%%

/* Lexer mínimo: números, + - * / ( ) e '\n' */
int yylex(void) {
  int c = getchar();
  while (c==' ' || c=='\t' || c=='\r') c = getchar();   /* ignora espaços */

  if (c == EOF) return 0;        /* 0 = EOF */
  if (c == '\n' || c=='+' || c=='-' || c=='*' || c=='/' || c=='(' || c==')') return c;

  if (c >= '0' && c <= '9') {    /* número inteiro */
    int val = 0;
    do { val = val*10 + (c - '0'); c = getchar(); } while (c >= '0' && c <= '9');
    yylval = val;
    ungetc(c, stdin);
    return NUM;
  }

  yyerror("caractere invalido");
  return 0;
}

void yyerror(const char *s) { fprintf(stderr, "Erro de sintaxe: %s\n", s); }
int main(void) { return yyparse(); }
EOF

# 3) Gerar, compilar e testar
bison -d -v calc.y
gcc calc.tab.c -o calc

echo "== Entradas =="
printf "2+3*4\n(2+3)*4\n2+*3\n" | ./calc || true


== Entradas ==
14
20


Erro de sintaxe: syntax error, unexpected '*', expecting NUM or '('


Aula 4 - Unindo Flex + Bison

In [8]:
#@title Flex + Bison no Colab — célula única (instalar, compilar e testar)
%%bash
set -e

# 1) Instalar ferramentas (silencioso)
apt-get -qq update 2>/dev/null
apt-get -qq install -y flex bison build-essential >/dev/null 2>&1

# 2) Bison: calc.y (parser de expressões)
cat > calc.y << 'EOF'
%{
  #include <stdio.h>
  #include <stdlib.h>
  void yyerror(const char *s);
  int yylex(void);
%}

%define parse.error verbose           /* mensagens: unexpected '...', expecting ... */
%define api.value.type {int}
%token NUM
%left '+' '-'
%left '*' '/'
%start input

%%
/* várias linhas; cada linha pode ter uma expressão */
input
  : /* vazio */
  | input '\n'
  | input expr '\n'          { printf("%d\n", $2); }
  ;

expr
  : expr '+' expr            { $$ = $1 + $3; }
  | expr '-' expr            { $$ = $1 - $3; }
  | expr '*' expr            { $$ = $1 * $3; }
  | expr '/' expr            { if ($3==0) { yyerror("divisão por zero"); YYERROR; } else $$ = $1 / $3; }
  | '(' expr ')'             { $$ = $2; }
  | NUM                      { $$ = $1; }
  ;
%%

void yyerror(const char *s) { fprintf(stderr, "Erro de sintaxe: %s\n", s); }
int main(void) { return yyparse(); }
EOF

# 3) Flex: lex.l (lexer simples)
cat > lex.l << 'EOF'
%option noyywrap
%{
  #include <stdlib.h>
  #include "calc.tab.h"   /* tokens e yylval */
%}

/* padrões */
WS   [ \t\r]+
NUM  [0-9]+

%%
{WS}            ;                 /* ignora espaços */
\n              return '\n';      /* cada linha é uma expressão */
"+"             return '+';
"-"             return '-';
"*"             return '*';
"/"             return '/';
"("             return '(';
")"             return ')';
{NUM}           { yylval = atoi(yytext); return NUM; }
.               { return yytext[0]; }  /* caractere inesperado vai ao parser */
%%
EOF

# 4) Gerar e compilar
bison -d -v calc.y
flex lex.l
gcc lex.yy.c calc.tab.c -o calc

# 5) Testes rápidos (2 válidos, 1 inválido)
echo "== Entradas =="
printf "2+3*4\n(2+3)*4\n2+*3\n" | tee /dev/stderr >/dev/null
printf "2+3*4\n(2+3)*4\n2+*3\n" | ./calc || true


== Entradas ==
14
20


2+3*4
(2+3)*4
2+*3
Erro de sintaxe: syntax error, unexpected '*', expecting NUM or '('
